In [1]:
import requests

In [62]:
%%time
#Latitude,Longitude
latlonglist = ["-83.738798,43.018999"] 

longstr = ''
for i, c in enumerate(latlonglist):
    if i == 0:
        longstr = c
    else:
        longstr += ';' + c

ip_add = "52.90.54.161"
init_lat_long = "-83.703084,43.018999"
how = "driving" #"walking" or "driving"
url = f"http://{ip_add}:5000/table/v1/{how}/{init_lat_long};{longstr}"
print(url)
r = requests.get(url)
output = r.json()
#output['durations'] # output is in seconds
output['durations']

http://52.90.54.161:5000/table/v1/driving/-83.703084,43.018999;-83.738798,43.018999
CPU times: user 3.87 ms, sys: 6.48 ms, total: 10.4 ms
Wall time: 311 ms


[[0, 520.9], [534.3, 0]]